In [1]:
#load packages
import numpy as np
import pandas as pd
import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, GRU, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
print('Keras version:' + keras.__version__)
print('Tensorflow version:' + tensorflow.__version__)

Keras version:2.2.4
Tensorflow version:1.12.0


In [3]:
nfeat, kernel, timeStep, length, fs = 36, 8, 20, 100, 100

In [4]:
#data (dummy)
data = np.random.rand(length*fs,nfeat)
classes = 0*data[:,0]
classes[:int(length/2*fs)] = 1


In [5]:
classes.shape

(10000,)

In [9]:
#splitting matrix 
#data
X = np.asarray([data[i*timeStep:(i + 1)*timeStep,:] for i in range(0,length * fs // timeStep)])
#classes
#Y = np.asarray([classes[i:(i + 1)*timeStep] for i in range(0,length * fs // timeStep)])
Y = classes[::timeStep] 

In [11]:
Y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [18]:
#split into training and test set
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2,random_state=0)

In [19]:
#one-hot-encoding
trainY_OHC = np_utils.to_categorical(trainY)
trainY_OHC.shape, trainX.shape

((400, 2), (400, 20, 36))

In [23]:
#set up model with simple 1D convnet
model = Sequential()
model.add(Conv1D(8,16,activation='relu',input_shape=(timeStep,nfeat)))
model.add(MaxPooling1D(4))
model.add(Flatten())
model.add(Dense(6,activation='tanh'))
model.add(Dense(2,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 5, 8)              4616      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 8)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 14        
Total params: 4,684
Trainable params: 4,684
Non-trainable params: 0
_________________________________________________________________


In [24]:
#compile model
model.compile(loss='mse',optimizer='Adam' ,metrics=['accuracy'])

In [25]:
#train model
model.fit(trainX,trainY_OHC,epochs=5,batch_size=4,validation_split=0.2)

Train on 320 samples, validate on 80 samples
Epoch 1/5
320/320 [==============================] - 1s 2ms/step - loss: 0.2528 - acc: 0.4781 - val_loss: 0.2496 - val_acc: 0.5000
Epoch 2/5
320/320 [==============================] - 0s 622us/step - loss: 0.2512 - acc: 0.5094 - val_loss: 0.2527 - val_acc: 0.4875
Epoch 3/5
320/320 [==============================] - 0s 660us/step - loss: 0.2525 - acc: 0.4781 - val_loss: 0.2567 - val_acc: 0.4625
Epoch 4/5
320/320 [==============================] - 0s 596us/step - loss: 0.2503 - acc: 0.4813 - val_loss: 0.2503 - val_acc: 0.4875
Epoch 5/5
320/320 [==============================] - 0s 614us/step - loss: 0.2537 - acc: 0.4969 - val_loss: 0.2501 - val_acc: 0.4875
